In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..", os.pardir))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import test_config, setup


config_data = test_config.get_config_data()

setup.initialize_db(config_data["database"])


In [ ]:
from src import pipeline
from src.utils import common


config_data = test_config.get_config_data()

responses = []
for scenerio_data in config_data["test"]["scenerios"]:
    pipeline.initialize_ltm_pipeline(config_data, scenerio_data)
    # pipeline.initialize_stm_pipeline(config_data, scenerio_data)

    pipeline.run_memory_creation_pipeline(config_data, scenerio_data)
    # pipeline.run_memory_consolidation_pipeline(config_data, scenerio_data)

    scenerio_responses = []
    for _ in range(config_data["test"]["num_replies_per_scenerio"]):
        response = pipeline.run_chat_pipeline(config_data, scenerio_data)
        scenerio_responses += [response]

    responses += [scenerio_responses]

common.save_jsonl_file(config_data["test"]["test_save_output_filepath"], responses)
